In [3]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from dotenv import load_dotenv
import os

# Load variables from .env file
load_dotenv()

api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
ts = TimeSeries(key = api_key, output_format = "pandas")

# Defining get asset data variable
def get_asset_data(symbol):
    data, _ = ts.get_daily(symbol = symbol, outputsize = "full")
    data = data[['4. close']] # DF becoems just the close column
    data.columns = [symbol] 
    data.index = pd.to_datetime(data.index) # Converts index from data strings to date/time objects
    return data.sort_index() 


# Get asset data for 3 assets
spy = get_asset_data("SPY")
ief = get_asset_data("IEF")
gld = get_asset_data("GLD")

# Merge into one DataFrame
assets = pd.concat([spy, ief, gld], axis = 1).dropna()

assets.tail()


,SPY,IEF,GLD
date,,,
2025-07-21,628.77,94.73,313.13
2025-07-22,628.86,95.00,316.10
2025-07-23,634.21,94.66,312.18
2025-07-24,634.42,94.52,310.27
2025-07-25,637.10,94.74,307.40


In [4]:
# Calculate 21 day trailing return (trading month)
lookback_days = 21

trailing_returns = assets.pct_change(periods = lookback_days)

trailing_returns.tail()

,SPY,IEF,GLD
date,,,
2025-07-21,0.052440,0.002010,0.009250
2025-07-22,0.058188,0.004228,0.019250
2025-07-23,0.056752,-0.002844,0.003536
2025-07-24,0.045552,-0.007768,0.013325
2025-07-25,0.049381,-0.005876,0.000912
